## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Leh,IN,34.17,77.58,59.02,37,0,4.09,Clear
1,1,Key Largo,US,25.09,-80.45,84.99,1,100,3.00,Clouds
2,2,Lebu,CL,-37.62,-73.65,51.57,93,79,6.22,Clouds
3,3,Chuy,UY,-33.70,-53.46,60.37,90,97,4.68,Clouds
4,4,Ancud,CL,-41.87,-73.82,52.00,81,90,12.75,Clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("what is the minimum temperature you would like for your trip? "))
max_temp = float(input("what is the maximum temperature you would like for your trip? "))

what is the minimum temperature you would like for your trip? 75
what is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(500)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Key Largo,US,25.09,-80.45,84.99,1,100,3.00,Clouds
6,6,Yellowknife,CA,62.46,-114.35,82.40,51,75,16.11,Clouds
8,8,Georgetown,MY,5.41,100.34,82.40,78,20,4.70,Clouds
9,9,Lufilufi,WS,-13.87,-171.60,81.70,80,73,11.43,Clouds
10,10,Sola,VU,-13.88,167.55,84.20,74,75,8.05,Clouds
...,...,...,...,...,...,...,...,...,...,...
695,695,Xingcheng,CN,40.62,120.72,79.57,79,0,7.29,Clear
698,698,Srikakulam,IN,18.30,83.90,82.36,82,100,5.28,Clouds
703,703,Eyl,SO,7.98,49.82,80.71,60,5,25.03,Clear
705,705,Antequera,ES,37.02,-4.56,90.00,19,0,11.41,Clear


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                254
City                   254
Country                254
Lat                    254
Lng                    254
Max Temp               254
Humidity               254
Cloudiness             254
Wind Speed             254
Current Description    254
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = preferred_cities_df.dropna()
clean_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Key Largo,US,25.09,-80.45,84.99,1,100,3.00,Clouds
6,6,Yellowknife,CA,62.46,-114.35,82.40,51,75,16.11,Clouds
8,8,Georgetown,MY,5.41,100.34,82.40,78,20,4.70,Clouds
9,9,Lufilufi,WS,-13.87,-171.60,81.70,80,73,11.43,Clouds
10,10,Sola,VU,-13.88,167.55,84.20,74,75,8.05,Clouds
...,...,...,...,...,...,...,...,...,...,...
695,695,Xingcheng,CN,40.62,120.72,79.57,79,0,7.29,Clear
698,698,Srikakulam,IN,18.30,83.90,82.36,82,100,5.28,Clouds
703,703,Eyl,SO,7.98,49.82,80.71,60,5,25.03,Clear
705,705,Antequera,ES,37.02,-4.56,90.00,19,0,11.41,Clear


In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Key Largo,US,84.99,Clouds,25.09,-80.45,
6,Yellowknife,CA,82.40,Clouds,62.46,-114.35,
8,Georgetown,MY,82.40,Clouds,5.41,100.34,
9,Lufilufi,WS,81.70,Clouds,-13.87,-171.60,
10,Sola,VU,84.20,Clouds,-13.88,167.55,
15,Albany,US,84.00,Rain,42.60,-73.97,
16,Chauk,MM,83.03,Rain,20.88,94.82,
21,Gushikawa,JP,87.80,Clouds,26.35,127.87,
22,Atuona,PF,78.84,Clear,-9.80,-139.03,
23,Bastia,FR,80.01,Clouds,42.67,9.33,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Key Largo,US,84.99,Clouds,25.09,-80.45,Bayside Inn Key Largo
6,Yellowknife,CA,82.40,Clouds,62.46,-114.35,The Explorer Hotel
8,Georgetown,MY,82.40,Clouds,5.41,100.34,Cititel Penang
9,Lufilufi,WS,81.70,Clouds,-13.87,-171.60,Leuaina Beach Resort and Spa
10,Sola,VU,84.20,Clouds,-13.88,167.55,Leumerus Bungalows
15,Albany,US,84.00,Rain,42.60,-73.97,
16,Chauk,MM,83.03,Rain,20.88,94.82,Bo Khin Guest House
21,Gushikawa,JP,87.80,Clouds,26.35,127.87,Kirari Guest House
22,Atuona,PF,78.84,Clear,-9.80,-139.03,Villa Enata
23,Bastia,FR,80.01,Clouds,42.67,9.33,Hotel Santa Maria


In [24]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace(" ", np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Key Largo,US,84.99,Clouds,25.09,-80.45,Bayside Inn Key Largo
6,Yellowknife,CA,82.40,Clouds,62.46,-114.35,The Explorer Hotel
8,Georgetown,MY,82.40,Clouds,5.41,100.34,Cititel Penang
9,Lufilufi,WS,81.70,Clouds,-13.87,-171.60,Leuaina Beach Resort and Spa
10,Sola,VU,84.20,Clouds,-13.88,167.55,Leumerus Bungalows
16,Chauk,MM,83.03,Rain,20.88,94.82,Bo Khin Guest House
21,Gushikawa,JP,87.80,Clouds,26.35,127.87,Kirari Guest House
22,Atuona,PF,78.84,Clear,-9.80,-139.03,Villa Enata
23,Bastia,FR,80.01,Clouds,42.67,9.33,Hotel Santa Maria
37,Narasannapeta,IN,82.00,Clouds,18.42,84.05,G.lingaraju home


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))